# 4.6 使用堆疊法處理分類問題

In [2]:
# --- 第 1 部分 ---
# 載入函式庫與資料集
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn import metrics
import numpy as np
bc = load_breast_cancer()

train_x, train_y = bc.data[:400], bc.target[:400]
test_x, test_y = bc.data[400:], bc.target[400:]


In [3]:
# --- 第 2 部分 ---
# 建立基學習器與超學習器
# 將基學習器放到串列中
base_learners = []

knn = KNeighborsClassifier(n_neighbors=2)
base_learners.append(knn)

dtr = DecisionTreeClassifier(max_depth=4, random_state=2)
base_learners.append(dtr)

mlpc = MLPClassifier(hidden_layer_sizes =(100, ), random_state=2)
base_learners.append(mlpc)

meta_learner = LogisticRegression() # 超學習器為邏輯斯迴歸

In [4]:
# --- 第 3 部分 ---
# 產生訓練超學習器用的中繼資料

# 建立變數以儲存中繼資料及其標籤
meta_data = np.zeros((len(base_learners), len(train_x)))  # meta_data.shape (3, 400)
meta_targets = np.zeros(len(train_x))                     # meta_targets.shape (400,) 

# 進行交叉驗證
KF = KFold(n_splits=5) # 使用KFold函式並指訂將資料分成5折，傳回這5折資料的索引
index = 0
for train_indices, test_indices in KF.split(train_x): # train_indices包含4/5的資料索引 ,test_indices包含1/5的資料索引
    # 前K-1折是訓練資料集
    # 第K折是驗證資料集
    for i in range(len(base_learners)):
        learner = base_learners[i]
        
        learner.fit(train_x[train_indices], train_y[train_indices])
        p = learner.predict_proba(train_x[test_indices])[:,0] # predict_proba 只記錄每一筆資料屬於其中一個類別的機率

        meta_data[i][index:index + len(test_indices)] = p

    meta_targets[index:index + 
                 len(test_indices)] = train_y[test_indices]
    index += len(test_indices)

# 中繼資料的特徵存在meta_data陣列
# 中繼資料的標籤存在meta_targets陣列        
    
# 將中繼資料轉置為超學習器需要的形式，使陣列的每一列是一筆資料，每一行是一個基學習器的預測值
meta_data = meta_data.transpose() # meta_data.shape (400, 3)

C:\Users\011305\Anaconda3\envs\py37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
C:\Users\011305\Anaconda3\envs\py37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [5]:
# --- 第 4 部分 ---
# 產生超學習器的測試資料
test_meta_data = np.zeros((len(base_learners), len(test_x)))  # test_meta_data.shape  (3, 169)
base_acc = []
for i in range(len(base_learners)):
    b = base_learners[i]
    b.fit(train_x, train_y)
    predictions = b.predict_proba(test_x)[:,0] # predict_proba 只記錄每一筆資料屬於其中一個類別的機率
    test_meta_data[i] = predictions

    acc = metrics.accuracy_score(test_y, b.predict(test_x))
    base_acc.append(acc)
test_meta_data = test_meta_data.transpose()  # test_meta_data.shape  (169, 3)

C:\Users\011305\Anaconda3\envs\py37\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [6]:
# --- 第 5 部分 ---
# 訓練超學習器
meta_learner.fit(meta_data, meta_targets)
ensemble_predictions = meta_learner.predict(test_meta_data)

acc = metrics.accuracy_score(test_y, ensemble_predictions)


In [7]:
# --- 第 6 部分 ---
# 顯示結果
print('Acc  Name')
print('-'*20)
for i in range(len(base_learners)):
    learner = base_learners[i]
    print(f'{base_acc[i]:.2f} {learner.__class__.__name__}')
print(f'{acc:.2f} Ensemble')


Acc  Name
--------------------
0.86 KNeighborsClassifier
0.89 DecisionTreeClassifier
0.93 MLPClassifier
0.95 Ensemble
